In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

Load Data

In [2]:
loader = PyPDFLoader("\\Users\\wolfe\\OneDrive\\Desktop\\BOOKS\\calculus-early-transcendentals-8th-ed.pdf")

In [3]:
data = loader.load()

In [ ]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

Chunk data into smaller chunks

In [6]:
# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
print (f'Now you have {len(texts)} documents')

Create embeddings of your documents to get ready for semantic search

In [ ]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from tqdm.autonotebook import tqdm
import pinecone

In [9]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', '')

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [11]:
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "wolfe-test"

In [12]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [21]:
query = "How do I find the acceleration of my car?"
docs = docsearch.similarity_search(query)

In [ ]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:500])

Query those docs to get your answer back

In [13]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [14]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [23]:
query = "What's a good summary of parametric functions?"
docs = docsearch.similarity_search(query)

In [ ]:
chain.run(input_documents=docs, question=query)